## Project 5
### Load Data

In [1]:
%env BASE_DIR=/home/jupyter/snippets_10_13/snippets

env: BASE_DIR=/home/jupyter/snippets_10_13/snippets


In [21]:
!gsutil cp gs://cs327e-open-access/fs_shopify.zip .

Copying gs://cs327e-open-access/fs_shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [ ]:
!unzip fs_shopify.zip

Archive:  fs_shopify.zip
replace fs_shopify/apps.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!gsutil cp gs://cs327e-open-access/fs_samples.zip .

In [ ]:
!unzip fs_samples.zip

### Create Collections

In [2]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()

#### Created Firestore Collection "apps" and subcollections "pricing_plans", "key_benefits", and "reviews"

In [4]:
batch = db.batch()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
reviews_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

for apps_row in apps_rows:
    
    apps_record = {}
    apps_record['id'] = apps_row[0]
    apps_record['url'] = apps_row[1]
    apps_record['title'] = apps_row[2]
    apps_record['developer'] = apps_row[3]
    apps_record['developer_link'] = apps_row[4]
    apps_record['icon'] = apps_row[5]
    apps_record['rating'] = apps_row[6]
    apps_record['reviews_count'] = apps_row[7]
    
    apps_ref = db.collection('apps').document(apps_row[0])
    
    batch.set(apps_ref, apps_record)
    
    for pp_row in pp_rows:
        
        if apps_row[0] == pp_row[1]:
            
            pricing_plans_record = {}
            pricing_plans_record['id'] = pp_row[0]
            pricing_plans_record['title'] = pp_row[2]
            pricing_plans_record['price'] = pp_row[3]
            for ppf_row in ppf_rows:
        
                if pp_row[0] == ppf_row[1]:
            
                    pricing_plans_record['features'] = ppf_row[2]
                
            pricing_plans_ref = apps_ref.collection('pricing_plans').document(pp_row[0])
        
            batch.set(pricing_plans_ref, pricing_plans_record)
            
    for kb_row in kb_rows:
        
        if apps_row[0] == kb_row[0]:
            
            key_benefits_record = {}
            key_benefits_record['title'] = kb_row[1]
            key_benefits_record['description'] = kb_row[2]
            
            key_benefits_ref = apps_ref.collection('key_benefits').document(kb_row[1].replace("/", "_"))
        
            batch.set(key_benefits_ref, key_benefits_record)
            
    for reviews_row in reviews_rows:
        
        if apps_row[0] == reviews_row[0]:
            
            reviews_record = {}
            reviews_record['author'] = reviews_row[1]
            reviews_record['rating'] = reviews_row[2]
            reviews_record['posted_at'] = reviews_row[3]
            
            reviews_ref = apps_ref.collection('reviews').document(str(reviews_row[1]).replace("/", "_").replace(".", "_"))
        
            batch.set(reviews_ref, reviews_record)
    
    batch.commit()

#### Created Firestore Collection "category" and subcollection "apps"

In [8]:
batch = db.batch()

df = pd.read_csv('fs_shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/apps.csv', sep=',', header=0, lineterminator='\n')
apps_rows = df.values.tolist()

df = pd.read_csv('fs_shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
appscat_rows = df.values.tolist()

for cat_row in cat_rows:
    
    cat_record = {}
    cat_record['id'] = cat_row[0]
    cat_record['title'] = cat_row[1]
    
    cat_ref = db.collection('category').document(cat_row[0])
    
    batch.set(cat_ref, cat_record)
    
    for appscat_row in appscat_rows:
        
        if cat_row[0] == appscat_row[1]:
            
            for apps_row in apps_rows:
                
                if appscat_row[0] == apps_row[0]:
            
                    apps_record = {}
                    apps_record['id'] = apps_row[0]
                    apps_record['url'] = apps_row[1]
                    apps_record['title'] = apps_row[2]
                    apps_record['developer'] = apps_row[3]
                    apps_record['developer_link'] = apps_row[4]
                    apps_record['icon'] = apps_row[5]
                    apps_record['rating'] = apps_row[6]
                    apps_record['reviews_count'] = apps_row[7]
                
                    apps_ref = cat_ref.collection('apps').document(apps_row[0])
        
                    batch.set(apps_ref, apps_record)

    batch.commit()

### Retrieve the number of documents in each collection and subcollection using count().

In [43]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

db = firestore.Client()
apps_ref = db.collection('apps')
query = apps_ref.count()
results = query.get()

for result in results:
    print(f"Number of documents in the apps collection: {result[0].value}")

print()

from google.api_core.retry import Retry
results = apps_ref.stream(retry=Retry())
count_pricing_plans = 0
count_key_benefits = 0
count_reviews = 0

for result in results:
    
    pricing_plans_ref = apps_ref.document(result.id).collection('pricing_plans')
    query = pricing_plans_ref.count()
    tempq = query.get()
    
    for temp in tempq:
        
        count_pricing_plans = temp[0].value
        print(f"Number of documents in the pricing_plans subcollection of document {result.id} in the apps collection: {count_pricing_plans}")

results = apps_ref.stream(retry=Retry())
print()

for result in results:
    
    key_benefits_ref = apps_ref.document(result.id).collection('key_benefits')
    query = key_benefits_ref.count()
    tempq = query.get()
    
    for temp in tempq:
        
        count_key_benefits = temp[0].value
        print(f"Number of documents in the key_benefits subcollection of document {result.id} in the apps collection: {count_key_benefits}")

results = apps_ref.stream(retry=Retry())
print()

for result in results:   
    
    reviews_ref = apps_ref.document(result.id).collection('reviews')
    query = reviews_ref.count()
    tempq = query.get()
    
    for temp in tempq:
        
        count_reviews = temp[0].value
        print(f"Number of documents in the reviews subcollection of document {result.id} in the apps collection: {count_reviews}")


Number of documents in the apps collection: 3547

Number of documents in the pricing_plans subcollection of document 00289a9f-9f12-45b1-963b-67e78403f7c7 in the apps collection: 1
Number of documents in the pricing_plans subcollection of document 00312cea-3595-4d65-a60d-6e8e5b7d6897 in the apps collection: 2
Number of documents in the pricing_plans subcollection of document 003b3dbd-1295-48cb-a3e8-8f3a0e8fc6df in the apps collection: 3
Number of documents in the pricing_plans subcollection of document 004cc187-6def-4058-a7f3-78e6edfe2e3c in the apps collection: 1
Number of documents in the pricing_plans subcollection of document 005d6c7a-9eb9-4b35-bdd4-040d383e86cf in the apps collection: 1
Number of documents in the pricing_plans subcollection of document 0067a20e-e307-490e-acc5-8442d4bdca4b in the apps collection: 1
Number of documents in the pricing_plans subcollection of document 008ce864-6073-4b61-b133-265ea6afa893 in the apps collection: 1
Number of documents in the pricing_plans

In [44]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

db = firestore.Client()
cat_ref = db.collection('category')
query = cat_ref.count()
results = query.get()

for result in results:
    print(f"Number of documents in the category collection: {result[0].value}")

print()

results = cat_ref.stream()
count_apps = 0

for result in results:
    
    apps_ref = cat_ref.document(result.id).collection('apps')
    query = apps_ref.count()
    tempq = query.get()
    
    for temp in tempq:
        
        count_apps = temp[0].value
        print(f"Number of documents in the apps subcollection of document {result.id} in the category collection: {count_apps}")
        

Number of documents in the category collection: 12

Number of documents in the apps subcollection of document 26a72de0d02e0e4e5f615332d61a878e in the category collection: 133
Number of documents in the apps subcollection of document 30a930262efca129caafcd586bc7e6fe in the category collection: 83
Number of documents in the apps subcollection of document 30ea2315e910c5e5335de42d5e87dca5 in the category collection: 602
Number of documents in the apps subcollection of document 5eb4e29e50e3f178acc614236ed107f4 in the category collection: 957
Number of documents in the apps subcollection of document 64430ad2835be8ad60c59e7d44e4b0b1 in the category collection: 310
Number of documents in the apps subcollection of document 737ad50051083aa051d127a53b3ac0da in the category collection: 190
Number of documents in the apps subcollection of document 9fe78acd078fc030de72c896adc93a2d in the category collection: 251
Number of documents in the apps subcollection of document bd125b49ce6b0425d29e33cb8b49a4

### Access Pattern 1 Query
#### Returned the id, title, developer, rating, and reviews_count for the top 10 productivity apps ranked by rating.

In [42]:
from google.cloud import firestore
from google.cloud.firestore_v1.base_query import FieldFilter
from google.cloud.firestore_v1 import aggregation

db = firestore.Client()
cat_ref = db.collection('category')
query = cat_ref.where(filter=FieldFilter('title', '==', 'Productivity'))
results = query.stream()

fields = ['id','title','developer','rating','reviews_count']

for result in results:
    
    presults = (cat_ref.document(result.id).collection('apps')
                .order_by('rating', direction=firestore.Query.DESCENDING)
                .limit(10)).stream()
    
    for presult in presults:
        
        result_list = [presult.to_dict()[field] for field in fields]
        print(result_list)


['fe5ae45c-379c-42bf-be7d-b5f6e15fc13c', 'Customer Tags', 'Union Works Apps', 5.0, 21]
['f99bb1e3-f326-4f10-8901-491652e9809b', 'Order Tagger', 'Union Works Apps', 5.0, 68]
['f864e3bd-da0e-41dc-be65-984325331475', 'SilkRoad ‑ Facebook Auto Ads', 'SilkRoad', 5.0, 2]
['f5344e64-9cda-4d97-b198-2aaeb5170518', 'Xporter Data Export Tool', 'Modd Apps Inc.', 5.0, 223]
['ed77a32d-0fa3-458b-b639-e01ea7b78ec0', 'FraudBlock Fraud Prevention', 'ShopFox', 5.0, 6]
['e0e231d6-4988-4a8c-ad84-cf18d0f38738', 'Automation Fox', 'Automation Fox', 5.0, 1]
['d75fa395-3a4d-41c3-88d1-2f802c1e2411', 'AirPower', 'BaseGenius', 5.0, 1]
['d0be03ff-f74d-4fb5-8d2c-79541a1aea1b', 'Excelify', 'Excelify.io', 5.0, 178]
['c9a86f8c-817e-4bb5-bbdd-e0e669bd7816', 'UPC Code Manager', 'Ventures & Adventures', 5.0, 1]
['c7578030-19fd-42e1-a5a5-03c7861220aa', 'EzySlips ‑ Shipping Automation', 'Onjection Labs', 5.0, 219]


### Access Pattern 2 Query
#### Returned the id, title, developer, rating, and reviews_count of the top 10 apps ranked by the number of reviews.

In [45]:
db = firestore.Client()
apps_ref = db.collection('apps')
query = apps_ref.where(filter=FieldFilter('reviews_count', '>', 0)).order_by('reviews_count', direction=firestore.Query.DESCENDING).limit(10)
results = query.stream()

fields = ['id','title','developer','rating','reviews_count']

for result in results:
    
    result_list = [result.to_dict()[field] for field in fields]
    print(result_list)

['d9f142ee-b141-4dc4-9353-173db61d2eb0', 'Privy ‑ Exit Pop Ups & Email', 'Privy', 4.7, 23078]
['78ea0810-c008-4a4e-a82f-de0c790e3286', 'Free Shipping Bar', 'Hextom', 4.9, 8737]
['b88488b0-9912-44d3-b736-224c36f09d95', 'Sales Pop ‑ Popup Notification', 'CartKit', 4.8, 6905]
['e528a60e-94f8-4e92-80e2-5bc6013b8283', 'BEST Currency Converter', 'Grizzly Apps', 4.8, 5986]
['be2640c4-01b5-4d52-9f68-cae8c0734d0d', 'Recart FB Messenger Marketing', 'Recart', 4.8, 5596]
['70bff9e0-4316-4cc6-84ce-92fcd1bc6925', 'EU Cookie Bar ‑ Cookie GDPR', 'Booster Apps', 4.7, 5259]
['171816e2-27d4-4552-a65e-ab44a312fe04', 'Sales Pop Master ‑ Countdown', 'Autoketing', 4.8, 4931]
['9025eff0-d714-4df1-930f-43f5582979ad', 'Ultimate Sales Boost', 'Hextom', 4.8, 4847]
['404183ac-6d52-4960-85d2-8350f37c2f0b', 'Ads by Varinode', 'Varinode, Inc.', 4.9, 4245]
['0380c5ed-7b91-4d4b-9a4e-f6a1c50269dd', 'Quick Announcement Bar', 'Hextom', 4.9, 4237]
